In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
from jinja2._identifier import pattern
%matplotlib inline
import datetime
from datetime import timedelta
import re
# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Any results you write to the current directory are saved as output.
# Loading the dataset from github
data = pd.read_csv('main_task.xls')

# Info
data

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838


In [5]:
data.Reviews[1]

"[['Unique cuisine', 'Delicious Nepalese food'], ['07/06/2017', '06/19/2016']]"

In [6]:
data.columns

Index(['Restaurant_id', 'City', 'Cuisine Style', 'Ranking', 'Rating',
       'Price Range', 'Number of Reviews', 'Reviews', 'URL_TA', 'ID_TA'],
      dtype='object')

In [7]:
# для удобства изменим наименование столбцов
data.columns = ['restaurant_id', 'city', 'cuisine_style', 'ranking', 'rating',
       'price_range', 'number_of_reviews', 'reviews', 'URL_TA', 'ID_TA']

In [8]:
data.head(3)

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781


### 1. Обработка NAN
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, даже отсутствие информации может быть важным признаком!
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак


In [9]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['number_of_reviews']).astype('uint8')


In [10]:
data['Number_of_Reviews_isNAN']


0        0
1        0
2        0
3        0
4        0
        ..
39995    0
39996    0
39997    0
39998    0
39999    0
Name: Number_of_Reviews_isNAN, Length: 40000, dtype: uint8

In [11]:
# заполним NaN средним значением по городу
data.loc[data['number_of_reviews'].isnull(),['number_of_reviews']] = data.groupby(['city'])['number_of_reviews'].transform('mean')


### Признак cuisine_style

In [12]:
#Отсутствующие значения можно отнести в группу неопределенных видов кухонь
data['cuisine_style'] = data['cuisine_style'].fillna('Unknown')
print(f"Количетсво отсутствующих значений {data['cuisine_style'].isna().sum()}")

Количетсво отсутствующих значений 0


### Признак price_range

In [13]:
data['price_range'] = data['price_range'].fillna('Other')
print(f"Количетсво отсутствующих значений {data['price_range'].isna().sum()}")

Количетсво отсутствующих значений 0


### Признак reviews
для корректрой обработки дат, заменим nan значения пустыми списками

In [14]:
data['reviews'] = data['reviews'].fillna("[[], []]")
print(f"Количетсво отсутствующих значений {data['reviews'].isna().sum()}")

Количетсво отсутствующих значений 0


## 2. Обработка признаков

In [15]:
#проверяем типы и полноту данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   restaurant_id            40000 non-null  object 
 1   city                     40000 non-null  object 
 2   cuisine_style            40000 non-null  object 
 3   ranking                  40000 non-null  float64
 4   rating                   40000 non-null  float64
 5   price_range              40000 non-null  object 
 6   number_of_reviews        40000 non-null  float64
 7   reviews                  40000 non-null  object 
 8   URL_TA                   40000 non-null  object 
 9   ID_TA                    40000 non-null  object 
 10  Number_of_Reviews_isNAN  40000 non-null  uint8  
dtypes: float64(3), object(7), uint8(1)
memory usage: 3.1+ MB


In [16]:
data.nunique(dropna=False)

restaurant_id              11909
city                          31
cuisine_style               9008
ranking                    11936
rating                         9
price_range                    4
number_of_reviews           1490
reviews                    33516
URL_TA                     39980
ID_TA                      39980
Number_of_Reviews_isNAN        2
dtype: int64

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:

##### Label Encoding
##### One-Hot Encoding
##### Target Encoding
##### Hashing
Выбор кодирования зависит от признака и выбраной модели. Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:

для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)


### Признак City

In [17]:
#список уникальных городов
city_lst = data.city.value_counts().index.to_list()
city_lst

['London',
 'Paris',
 'Madrid',
 'Barcelona',
 'Berlin',
 'Milan',
 'Rome',
 'Prague',
 'Lisbon',
 'Vienna',
 'Amsterdam',
 'Brussels',
 'Hamburg',
 'Munich',
 'Lyon',
 'Stockholm',
 'Budapest',
 'Warsaw',
 'Dublin',
 'Copenhagen',
 'Athens',
 'Edinburgh',
 'Zurich',
 'Oporto',
 'Geneva',
 'Krakow',
 'Oslo',
 'Helsinki',
 'Bratislava',
 'Luxembourg',
 'Ljubljana']

In [18]:
data = pd.get_dummies(data, columns=[ 'city',], dummy_na=True)
data.head(3)

,restaurant_id,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,URL_TA,ID_TA,Number_of_Reviews_isNAN,...,city_Oporto,city_Oslo,city_Paris,city_Prague,city_Rome,city_Stockholm,city_Vienna,city_Warsaw,city_Zurich,city_nan
0,id_5569,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,0,...,0,0,1,0,0,0,0,0,0,0
1,id_1535,Unknown,1537.0,4.0,Other,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,0,...,0,0,0,0,0,1,0,0,0,0
2,id_352,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,0,...,0,0,0,0,0,0,0,0,0,0


### Признак price_range

In [19]:
# количество уникальных признаков
data['price_range'].value_counts()

$$ - $$$    18412
Other       13886
$            6279
$$$$         1423
Name: price_range, dtype: int64

In [20]:
print("заменим значения на последовательные числа 0=no values, 1=low price, 2=Middle price, 3=high price")
def price_range(item):
    item = item.apply(lambda x: 1 if x=='$' else 3 if x=='$$$$' else 2 if x=='$$ - $$$' else 0)
    return item

data['price_range'] = price_range(data['price_range'])
data['price_range'].unique() #Check


заменим значения на последовательные числа 0=no values, 1=low price, 2=Middle price, 3=high price


array([2, 0, 3, 1], dtype=int64)

In [21]:
#encoder = LabelEncoder()
#data['price_range'] = encoder.fit_transform(data['price_range'].dropna())

pp = data['price_range'].value_counts(sort=True)
print(f"самая популярная ценовая категория ресторанов {pp}")

самая популярная ценовая категория ресторанов 2    18412
0    13886
1     6279
3     1423
Name: price_range, dtype: int64


### Признак cuisine_style

In [22]:
# приводим признак к "рабочему виду"
def clean_smb(smb):

    smb = smb.strip('[]') # Отбрасываем скобки.
    smb = smb.replace("\'",'') # Убираем надоедливые кавычки '.
    smb = smb.split(", ") # Разбиваем строку по названиям кухонь.
    return smb

# удалим лишние кавычки и заменим кириллицу на латинницу
data['cuisine_style'] = data['cuisine_style'].map(lambda x: x.replace('е', 'e').replace('а', 'a'))

# функция очистки от лишних символов
data['cuisine_style'] = data['cuisine_style'].apply(clean_smb)



In [23]:
# список уникальных кухонь
cuisine_lst = []
for i in range(len(data.cuisine_style)):
    for j in range(len(data.cuisine_style[i])):
        item = data.cuisine_style[i][j].lstrip(" '").rstrip("' ")
        if item not in cuisine_lst:
            cuisine_lst.append(item)

print("в датасете {} уникальных кухонь".format(len(cuisine_lst)-1))   # кухни с неопределенным видом "Unknown" не входят в отдельный вид

в датасете 125 уникальных кухонь


In [24]:
#сколько кухонь предлагает каждый ресторан в отдельный признак
cuisine_qty = [len(j) if j is not None else 1 for j in data.cuisine_style]
data['cuisine_amount'] = cuisine_qty

In [25]:
data.head(3)

,restaurant_id,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,URL_TA,ID_TA,Number_of_Reviews_isNAN,...,city_Oslo,city_Paris,city_Prague,city_Rome,city_Stockholm,city_Vienna,city_Warsaw,city_Zurich,city_nan,cuisine_amount
0,id_5569,"[European, French, International]",5570.0,3.5,2,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,0,...,0,1,0,0,0,0,0,0,0,3
1,id_1535,[Unknown],1537.0,4.0,0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,0,...,0,0,0,0,1,0,0,0,0,1
2,id_352,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri...",353.0,4.5,3,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,0,...,0,0,0,0,0,0,0,0,0,7


In [26]:

cuisines = data.explode('cuisine_style') 

# создаем словарь с частотой каждого вида в датасете
freq_cuisines = cuisines.cuisine_style.value_counts().to_dict()#запись в словарь


print("Наиболее часто представлена кухня {}".format(max(freq_cuisines, key=freq_cuisines.get)))


Наиболее часто представлена кухня Vegetarian Friendly


In [27]:
freq_cuisines

{'Vegetarian Friendly': 11189,
 'European': 10060,
 'Unknown': 9283,
 'Mediterranean': 6277,
 'Italian': 5964,
 'Vegan Options': 4486,
 'Gluten Free Options': 4113,
 'Bar': 3297,
 'French': 3190,
 'Asian': 3011,
 'Pizza': 2849,
 'Spanish': 2798,
 'Pub': 2449,
 'Cafe': 2325,
 'Fast Food': 1705,
 'British': 1595,
 'International': 1584,
 'Seafood': 1505,
 'Japanese': 1464,
 'Central European': 1393,
 'American': 1315,
 'Sushi': 1156,
 'Chinese': 1145,
 'Portuguese': 1107,
 'Indian': 1041,
 'Middle Eastern': 782,
 'Thai': 743,
 'Wine Bar': 697,
 'German': 662,
 'Healthy': 620,
 'Greek': 604,
 'Halal': 598,
 'Czech': 595,
 'Fusion': 577,
 'Steakhouse': 573,
 'Barbecue': 555,
 'Contemporary': 523,
 'Vietnamese': 513,
 'Eastern European': 496,
 'Soups': 494,
 'Grill': 490,
 'Gastropub': 471,
 'Mexican': 445,
 'Turkish': 444,
 'Delicatessen': 392,
 'Austrian': 380,
 'South American': 372,
 'Polish': 365,
 'Hungarian': 352,
 'Scandinavian': 342,
 'Lebanese': 329,
 'Latin': 302,
 'Diner': 295,


## Признак 'Reviews'



In [28]:
import re
# разделим данные в столбце на два списка
data[['review_text','review_date']] = pd.DataFrame(data['reviews'].str.split("],",1).to_list())
data[['review_text']] = data['review_text'].apply(lambda x: x[2:])
data[['review_date']] = data['review_date'].apply(lambda x: x[2:-2])


In [29]:
data[['review_text', 'review_date' ]].sample()

,review_text,review_date
30644,"'average', 'OK'","'08/25/2017', '07/21/2017'"


In [30]:
# попробуем проанализировать текст отзывов

words = {'will come back':4,"WOW":4,"fantastic":4,"Fantastic":4,"delightful":3,"great":3,"Great":3,"Delightful":3,"yummi":3,"Yummi":3,"nice":2,"nice":2,'good':1, 'Good':1,"Average":0,"average":0,"bad":-1,'Bad':-1,"disgusting":-2,"avoid":-3,"Avoid":-3,"worst":-4,"Worst":-4}


def review_analysis_column(x,y):
    result = 0
    for word in y:
        if re.search(r'\b%s\b' % word, x) is not None:
            result = y[word]
    return result

data['review_analysis'] = data.review_text.apply(lambda x: review_analysis_column(x, words))
data['review_analysis'].value_counts()

 0    23897
 1     7774
 3     5761
 2     1114
-1      533
 4      514
-4      245
-3      144
-2       18
Name: review_analysis, dtype: int64

In [31]:
# разделим даты отзывов на два признака
data[['review_1','review_2']] = pd.DataFrame(data['review_date'].str.split(", ",1).to_list())

In [32]:
#заполним пропуски 
data[['review_2']] = data['review_2'].fillna('0')

In [33]:
# переведем даты в формат datetime

def date_time(s):
    #"""функция для перевода в формат datetime"""
    date1 = re.search(r'\d{2}/\d{2}/\d{4}',s)
    if date1 is not None:
        return (datetime.datetime.strptime(date1.group(),'%m/%d/%Y').date())
    else:
        return None

In [34]:
#создадим столбец 'time_difference' - разница между отзывами
data[['review_1']] = data['review_1'].apply(date_time)
data[['review_2']] = data['review_2'].apply(date_time)
time_difference = abs(data['review_1'] - data['review_2'])
data['time_difference'] = time_difference.dropna().apply(lambda x: abs(np.timedelta64(x).astype(int)))
data[['time_difference']] = data['time_difference'].fillna(0)

In [35]:
data.loc[:1]

,restaurant_id,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,URL_TA,ID_TA,Number_of_Reviews_isNAN,...,city_Warsaw,city_Zurich,city_nan,cuisine_amount,review_text,review_date,review_analysis,review_1,review_2,time_difference
0,id_5569,"[European, French, International]",5570.0,3.5,2,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,0,...,0,0,0,3,"'Good food at your doorstep', 'A good hotel re...","'12/31/2017', '11/20/2017'",1,2017-12-31,2017-11-20,9.480192e+08
1,id_1535,[Unknown],1537.0,4.0,0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,0,...,0,0,0,1,"'Unique cuisine', 'Delicious Nepalese food'","'07/06/2017', '06/19/2016'",0,2017-07-06,2016-06-19,2.023670e+09


In [36]:
#переведем столбец ID_TA в цифровой формат
data[['ID_TA']] = data['ID_TA'].dropna().apply(lambda x: int(x[1:]))
data[['ID_TA']] = data['ID_TA'].fillna(0) 

# **Feature Engineering**

In [37]:
#Заменим наименования видов кухонь на значения словаря с их нормализованной частотой и запишем их в новый признак

data = data.explode('cuisine_style')


freq_cuisines = data.cuisine_style.value_counts(normalize=True).to_dict()


data['cuisine_code_freq'] = data['cuisine_style'].apply(lambda v: freq_cuisines[v],freq_cuisines.values())
data.loc[:1] 

,restaurant_id,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,URL_TA,ID_TA,Number_of_Reviews_isNAN,...,city_Zurich,city_nan,cuisine_amount,review_text,review_date,review_analysis,review_1,review_2,time_difference,cuisine_code_freq
0,id_5569,European,5570.0,3.5,2,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,1912643,0,...,0,0,3,"'Good food at your doorstep', 'A good hotel re...","'12/31/2017', '11/20/2017'",1,2017-12-31,2017-11-20,9.480192e+08,0.095905
0,id_5569,French,5570.0,3.5,2,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,1912643,0,...,0,0,3,"'Good food at your doorstep', 'A good hotel re...","'12/31/2017', '11/20/2017'",1,2017-12-31,2017-11-20,9.480192e+08,0.030411
0,id_5569,International,5570.0,3.5,2,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,1912643,0,...,0,0,3,"'Good food at your doorstep', 'A good hotel re...","'12/31/2017', '11/20/2017'",1,2017-12-31,2017-11-20,9.480192e+08,0.015101
1,id_1535,Unknown,1537.0,4.0,0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,7992032,0,...,0,0,1,"'Unique cuisine', 'Delicious Nepalese food'","'07/06/2017', '06/19/2016'",0,2017-07-06,2016-06-19,2.023670e+09,0.088497


In [38]:
data.columns

Index(['restaurant_id', 'cuisine_style', 'ranking', 'rating', 'price_range',
       'number_of_reviews', 'reviews', 'URL_TA', 'ID_TA',
       'Number_of_Reviews_isNAN', 'city_Amsterdam', 'city_Athens',
       'city_Barcelona', 'city_Berlin', 'city_Bratislava', 'city_Brussels',
       'city_Budapest', 'city_Copenhagen', 'city_Dublin', 'city_Edinburgh',
       'city_Geneva', 'city_Hamburg', 'city_Helsinki', 'city_Krakow',
       'city_Lisbon', 'city_Ljubljana', 'city_London', 'city_Luxembourg',
       'city_Lyon', 'city_Madrid', 'city_Milan', 'city_Munich', 'city_Oporto',
       'city_Oslo', 'city_Paris', 'city_Prague', 'city_Rome', 'city_Stockholm',
       'city_Vienna', 'city_Warsaw', 'city_Zurich', 'city_nan',
       'cuisine_amount', 'review_text', 'review_date', 'review_analysis',
       'review_1', 'review_2', 'time_difference', 'cuisine_code_freq'],
      dtype='object')

In [39]:
data = data.drop(columns=['restaurant_id','cuisine_style','reviews', 'URL_TA','review_text', 'review_date', 'review_1', 'review_2',])
data.head(3)

,ranking,rating,price_range,number_of_reviews,ID_TA,Number_of_Reviews_isNAN,city_Amsterdam,city_Athens,city_Barcelona,city_Berlin,...,city_Rome,city_Stockholm,city_Vienna,city_Warsaw,city_Zurich,city_nan,cuisine_amount,review_analysis,time_difference,cuisine_code_freq
0,5570.0,3.5,2,194.0,1912643,0,0,0,0,0,...,0,0,0,0,0,0,3,1,948019200.0,0.095905
0,5570.0,3.5,2,194.0,1912643,0,0,0,0,0,...,0,0,0,0,0,0,3,1,948019200.0,0.030411
0,5570.0,3.5,2,194.0,1912643,0,0,0,0,0,...,0,0,0,0,0,0,3,1,948019200.0,0.015101


In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104896 entries, 0 to 39999
Data columns (total 42 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   ranking                  104896 non-null  float64
 1   rating                   104896 non-null  float64
 2   price_range              104896 non-null  int64  
 3   number_of_reviews        104896 non-null  float64
 4   ID_TA                    104896 non-null  int64  
 5   Number_of_Reviews_isNAN  104896 non-null  uint8  
 6   city_Amsterdam           104896 non-null  uint8  
 7   city_Athens              104896 non-null  uint8  
 8   city_Barcelona           104896 non-null  uint8  
 9   city_Berlin              104896 non-null  uint8  
 10  city_Bratislava          104896 non-null  uint8  
 11  city_Brussels            104896 non-null  uint8  
 12  city_Budapest            104896 non-null  uint8  
 13  city_Copenhagen          104896 non-null  uint8  
 14  city_